# Recuperación de pacientes del archivo de XiO
---------
### Tests locales
## Procedimiento
- Configurar las herramientas necesarias
- Recuperar mediante FTP los datos archivados de XiO (herramientas externas)
- Subir a **gDrive** de Radiofísica los datos archivados
- Introducir los parámetros relevantes
- Correr los scripts de recuparación
- Descargar los archivos recuperados
- Limpiar en gDrive los archivos creados
## Configuración de herramientas

In [1]:
# Importación de módulos
# - Manipulación de archivos DICOM
import pydicom as dicom
# - Conversión del formato nativo de XiO a DICOM
import pyplastimatch as plast
# - Rutas y comprobaciones de existencia de ficheros 
import os 
from pathlib import Path
from glob import glob
from sys import path as syspath

In [2]:
%cd /home/radiofisica/Shares/Radiofisica/csr/XiO

/home/radiofisica/Shares/Radiofisica/csr/XiO


In [3]:
from XiORecovery.scripts.dcmcoher import correctImagePositionPatientInCTImages, correctRTPlan

Crear uan referencia al directorio base de trabajo, crearlo e ir a él

In [4]:
%%capture
# Referenciar
baseworkdir = '/home/radiofisica/devel'
baseworkdir = os.path.join(baseworkdir, 'patient')

# Crear el directorio
if os.path.exists(baseworkdir) is False:
    os.makedirs(baseworkdir)

# Ir al directorio base
%cd $baseworkdir

Crear la carpeta `scripts` en el directorio base de trabajo en **gDrive** y descargar los scripts que complementan el funcionamiento de `plastimatch`

In [5]:
%%capture
# Descargar los scripts de corrección
if Path(baseworkdir + '/scripts').exists():
  !rm -rf $baseworkdir/scripts
!mkdir -p $baseworkdir/scripts
!wget --directory-prefix $baseworkdir/scripts https://raw.githubusercontent.com/csarux/XiOrecovery/main/scripts/xioss
!wget --directory-prefix $baseworkdir/scripts https://raw.githubusercontent.com/csarux/XiOrecovery/main/scripts/genindex
!wget --directory-prefix $baseworkdir/scripts https://raw.githubusercontent.com/csarux/XiOrecovery/main/scripts/deltact
!wget --directory-prefix $baseworkdir/scripts https://raw.githubusercontent.com/csarux/XiOrecovery/main/scripts/dcmcoher.py


# Añadir al sistema la ruta de los scripts
if baseworkdir + '/scripts' not in syspath:
  syspath.insert(0, baseworkdir + '/scripts')

# Comprobar si el plan genérico con un único campo está y si no descargarlo
if not Path('RTPlan.dcm').exists():
  !wget https://raw.githubusercontent.com/csarux/XiOrecovery/main/DICOMSets/RTPlan.dcm

In [6]:
ArchivedPatient = '154435_ORTEGA_FORTE_MARGARITA_Jun_01_2010.tgz'

In [7]:
!tar -xvzf /home/radiofisica/Shares/Radiofisica/csr/XiO/patient/$ArchivedPatient

154435_ORTEGA_FORTE_MARGARITA_Jun_01_2010.tgz
./archive_configuration
./patient/154435/
./patient/154435/.dirlock
./patient/154435/.focalts
./patient/154435/dvh/
./patient/154435/demographic
./patient/154435/plan/
./patient/154435/plan/1/
./patient/154435/plan/1/dose.1
./patient/154435/plan/1/drr.4.high
./patient/154435/plan/1/tel.1
./patient/154435/plan/1/drr.2.high
./patient/154435/plan/1/tel.1.dose.2
./patient/154435/plan/1/tel.1.dose.1
./patient/154435/plan/1/drr.3.high
./patient/154435/plan/1/drr.1.high
./patient/154435/plan/1/plan
./patient/154435/plan/1/tel.1.dose.3
./patient/154435/plan/1/tel.1.dose.4
./patient/154435/plan/1/dirdosevalid
./patient/154435/plan/PLANSETUP/
./patient/154435/plan/PLANSETUP/tel.1.dose.5
./patient/154435/plan/PLANSETUP/dose.1
./patient/154435/plan/PLANSETUP/tel.1
./patient/154435/plan/PLANSETUP/tel.1.dose.6
./patient/154435/plan/PLANSETUP/tel.1.dose.1
./patient/154435/plan/PLANSETUP/drr.1.high
./patient/154435/plan/PLANSETUP/plan
./patient/154435/plan

In [8]:
PatientID = ArchivedPatient.split('_')[0]
PatientID

'154435'

In [9]:
demographicfile = !find ./patient/$PatientID -name demographic -print 
with open(demographicfile[0]) as demogf:
  demographicdata = demogf.read()

ApellidosNombre = demographicdata.split('\n')[2]
ApellidosNombre

'ORTEGA FORTE,MARGARITA'

In [10]:
!tree -d patient

patient
└── 154435
    ├── anatomy
    │   └── studyset
    │       └── CT1
    │           ├── DCMData
    │           └── surfaces
    ├── dvh
    └── plan
        ├── 1
        └── PLANSETUP

10 directories


In [11]:
CT = 'CT1'
Plan ='1'

## Correr los scripts de recuperación
**Estrategia**

- Crear el índice de las imágenes de CT si no existe
- Recuperar las estructuras a partir de los archivos de texto `.msh` 
- Transformar estructuras y dosis a formato DICOM
- Introducir un plan básico genérico y asociarle a la dosis recpuerada y al conjunto de estructuras
- Establecer una marco de referencia coherente entre las imágenes CT y el conjunto plan, dosis, estructuras

Scripts de recuperación. Plastimatch separado por CT, estructura y dosis. Más seguro que tirar de la interpretación que hace plastimatch del directorio XiO.

In [12]:
# Si no existe crear el índice de imágenes CT y las máscaras `.mha` de todas las estructuras
if not Path(baseworkdir + '/patient/' + PatientID + '/anatomy/studyset/' + CT + '/index.dat').exists():
  # Copiar el script para la creación del índice de cortes de CT
  !cp scripts/genindex patient/$PatientID/anatomy/studyset/$CT

  # Cambiar el directorio de trabajo
  %cd patient/$PatientID/anatomy/studyset/$CT

  # Adjudicar permisos de ejecución y crear el índice
  !chmod +x genindex 
  !./genindex

# Si no existe crear la carpeta de estructuras y las máscaras `.mha` de todas las estructuras
if not Path(baseworkdir + '/patient/' + PatientID + '/anatomy/studyset/' + CT + '/ss').exists():
  # Crear la carpeta para los archivos máscara con las estructuras
  !mkdir -p ./ss

  # Copiar el script para la creación de las estructuras
  !cp $baseworkdir/scripts/xioss .

  # Correr el script de recuperación de las estructuras
  !chmod +x xioss
  !./xioss --input $baseworkdir/patient/$PatientID --plan $Plan > xioss.log

  # Cambiar el directorio de trabajo al raíz de los pacientes
  %cd $baseworkdir/patient

  # Copiar el script para determinar el delta entre los marcos de referencia del estudio CT y el plan
  !cp $baseworkdir/scripts/deltact .

  # Adjudicar permisos de ejecución y calcular el delta
  !chmod +x deltact
  !./deltact --input $PatientID --studyset $CT 
  

# Regresar al directorio base
%cd $baseworkdir

# Variables de pyplastimatch
verbose = True
path_to_logfile_xiodcm = './' + PatientID + '_xiodcm.log'

# Recuperación del estudio CT
# Opciones de plastimatch convert
convert_args_rt ={'input' :  'patient/' + PatientID + '/anatomy/studyset/' + CT + '/DCMData' ,
                  'output-dicom' : 'xiodcm'} 

# Correr el script de conversión de los datos XiO en estudio DICOM                   
plast.convert(verbose=verbose, path_to_log_file=path_to_logfile_xiodcm, **convert_args_rt )

# Recuperación de las estructuras
# Opciones de plastimatch convert
convert_args_rt ={'input-prefix' :  'patient/' + PatientID + '/anatomy/studyset/' + CT + '/ss',
                  'output-dicom' : 'xiodcm'} 

# Correr el script de conversión de los datos XiO en estudio DICOM                   
plast.convert(verbose=verbose, path_to_log_file=path_to_logfile_xiodcm, **convert_args_rt )

# Recuperación de la dosis
# Opciones de plastimatch convert
convert_args_rt ={'input-dose-xio' :  'patient/' + PatientID + '/plan/' + Plan + '/dose.1',
                  'output-dicom' : 'xiodcm'} 

# Correr el script de conversión de los datos XiO en estudio DICOM                   
plast.convert(verbose=verbose, path_to_log_file=path_to_logfile_xiodcm, **convert_args_rt )

# Extracción del delta entre los marcos de referencia del estudio CT y el plan a partir del archivo CT#.infor
## Cambiar el directorio de trabajo al raíz de los pacientes
%cd $baseworkdir/patient

# Copiar el script para determinar el delta entre los marcos de referencia del estudio CT y el plan
!cp $baseworkdir/scripts/deltact .

## Asegurar que el archivo se actualiza
if Path('./deltaframes').exists():
  !rm -f ./deltaframes
## Adjudicar permisos de ejecución y calcular el delta
!chmod +x deltact
!./deltact --input $PatientID --studyset $CT > deltact.log

## Regresar al directorio base
%cd $baseworkdir

# Incorporar el plan genérico 
!cp RTPlan.dcm xiodcm

# Asociar al plan las dosis y estructuras
correctRTPlan()

# Establecer el mismo marco de referencia en todo el estudio
correctImagePositionPatientInCTImages(patientID=PatientID, studyset=CT)

mv: 'ss/patient.mha' and 'ss/patient.mha' are the same file
mv: 'ss/CTV.mha' and 'ss/CTV.mha' are the same file
mv: 'ss/PTV.mha' and 'ss/PTV.mha' are the same file
mv: 'ss/vejiga.mha' and 'ss/vejiga.mha' are the same file
Use of uninitialized value in string eq at ./xioss line 151.
Consistency lack in structure match between XiO contour file and plastimatch prefix output at ./xioss line 151.
/home/radiofisica/Development/patient/patient
Reading CT1.info file
Reference frame delta x[mm], y[mm]: -0.000001,-181.000000
Writing delta to deltaframes file
Finished!
/home/radiofisica/Development/patient

Running 'plastimatch convert' with the specified arguments:
  --input patient/154435/anatomy/studyset/CT1/DCMData
  --output-dicom xiodcm
... Done.

Running 'plastimatch convert' with the specified arguments:
  --input-prefix patient/154435/anatomy/studyset/CT1/ss
  --output-dicom xiodcm
... Done.

Running 'plastimatch convert' with the specified arguments:
  --input-dose-xio patient/154435/pl

Limpiar el directorio temporal de trabajo

In [22]:
!rm -rf 100668_xiodcm.log  archive_configuration  patient  RTPlan.dcm  scripts  xiodcm

In [13]:
# Asociar al plan las dosis y estructuras
correctRTPlan()

# Establecer el mismo marco de referencia en todo el estudio
correctImagePositionPatientInCTImages(patientID=PatientID, studyset=CT)